# imports

In [4]:
import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # relu, tanh # also included in nn package
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create network

In [5]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # (28x28) = 784 nodes
        super(NN, self).__init__() # call init of nn.Module
        self.fc1 = nn.Linear(input_size, 50) # 50 hidden nodes
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # no nonlinearity
        return x
    
model = NN(784, 10)
x = torch.randn(64, 784)
model(x).shape

torch.Size([64, 10])

In [6]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, 
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) # 28,28 -> 14,14
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, 
                               kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x
    
model = CNN()
x = torch.randn(64, 1, 28, 28)
model(x).shape

torch.Size([64, 10])

# hyperparameters

In [7]:
in_channels = 1
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 5

# load data

In [8]:
train_dataset = datasets.MNIST(root='dataset/', train=True,
                               transform=transforms.ToTensor(),
                               download=True) # training_set
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False,
                               transform=transforms.ToTensor(),
                               download=True) # training_set
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# initialize network

In [9]:
#model = NN(input_size=input_size, num_classes=num_classes).to(device)
model = CNN().to(device)

# loss and optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# train network

In [11]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("=> loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [12]:
for epoch in tqdm.tqdm(range(num_epochs)):
    losses = []

    # save checkpoint of the current state of the model
    checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint)

    for batch_idx, (data, targets) in enumerate(train_loader):
        # put data on device
        data = data.to(device)
        targets = targets.to(device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad() # set all gradients to 0 for each batch
                              # to not save gradient for each minibatch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  0%|          | 0/5 [00:00<?, ?it/s]/home/matt/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


=> saving checkpoint


 20%|██        | 1/5 [00:04<00:19,  4.92s/it]

=> saving checkpoint


 40%|████      | 2/5 [00:09<00:14,  4.72s/it]

...

 60%|██████    | 3/5 [00:14<00:09,  4.69s/it]

=> saving checkpoint


 80%|████████  | 4/5 [00:18<00:04,  4.66s/it]

=> saving checkpoint


100%|██████████| 5/5 [00:23<00:00,  4.68s/it]


# check accuracy

In [13]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    # don't need to compute gradients when checking accuracy
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, preds = scores.max(1) # index of maximum value for second dim
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58996 / 60000 with accuracy 98.33
Checking accuracy on test data
Got 9807 / 10000 with accuracy 98.07
